In [8]:
 # import necessary all library
import time
start = time.time()
import geopandas as gpd
import pandas as pd
from functools import reduce
import sys
'geopandas' in sys.modules
#!pip3 install matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import io
import pandas as pd
import networkx as nx
import osmnx as ox
import os 
from geopandas import GeoDataFrame, read_file
from geopandas.tools import overlay
import shapely.geometry as geom
import shapely.wkt
import numpy as np

In [9]:
# Read bike counter variables data
bdata=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/Eco_CA_Bike_data_circular_0.5_mile_buffer.csv')
bdata.head()

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,pct_at_least_college_education,HH_density,Median_HH_income,population_density,Median Age,employment_density,Total Commuter,Bike Commuter,Percentage of Bike Commuter,bike commuter density
0,671,37.791160,-122.459440,POLYGON ((-122.45030116607526 37.7911177158698...,5.153850,0.000000,0.797617,0.036196,0.527663,0.0,...,90.298608,3503.892064,152302.54020,7519.105739,38.966406,6219.810255,3127.241241,159.824309,5.110713,203.495292
1,1127,37.796165,-122.419490,POLYGON ((-122.4103506127677 37.79611961207228...,3.019345,0.002189,0.249510,0.165813,0.131352,0.0,...,82.603467,26537.520610,130355.30360,20802.530110,38.239858,19609.011290,11234.019440,197.352495,1.756740,251.277818
2,2415,37.774200,-122.436295,POLYGON ((-122.42715829014239 37.7741559374201...,4.642047,0.000000,0.687274,0.087677,0.155897,0.0,...,86.089770,17752.654100,142485.80360,29059.732560,34.641712,26625.663510,16134.096210,1351.356394,8.375780,1720.605996
3,2691,35.287736,-120.657826,POLYGON ((-120.64898426571617 35.2875646426652...,76.562372,0.008590,0.397234,0.009334,0.254896,0.0,...,84.855253,3235.253308,33837.50721,7407.807740,23.961926,7014.619758,3148.238608,400.995099,12.737125,510.564478
4,2692,35.287736,-120.657826,POLYGON ((-120.64898426571617 35.2875646426652...,76.562372,0.008590,0.397234,0.009334,0.254896,0.0,...,84.855253,3235.253308,33837.50721,7407.807740,23.961926,7014.619758,3148.238608,400.995099,12.737125,510.564478


In [10]:
bike_fac=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/Facilities_data/CA_Bike_Facilities- Bike_Manual_facilities.csv')
bike_fac=bike_fac.drop(['latitude','longitude'],axis=1)
bike_fac.head()

,Name,ID,name,highway,Bicycle_Fa
0,1,671,Bay Area Ridge Trail,Path,Path
1,2,1127,Broadway,primary,Bike Route
2,3,2415,Fell Street,primary,Bike Lane
3,4,2691,Halsey Circle,residential,Path
4,5,2692,Halsey Circle,residential,Path


In [11]:
bike_fac['Bicycle_Fa'].unique()

array(['Path', 'Bike Route', 'Bike Lane', 'Trail', 'Cycle Track'],
      dtype=object)

In [12]:
bdata.shape, bike_fac.shape

((416, 46), (414, 5))

In [13]:
bdata1=pd.merge(bdata, bike_fac, on=['ID'], how='inner')
bdata1.shape

(414, 50)

In [14]:
# add our MADBT after QC
madbt=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/our_bike_AASHTO_MADT for CA.csv')
madbt.head()

,Unnamed: 0,ID,year,month,AASHTO_MADT
0,0,671,2018,1,65.478571
1,1,671,2018,2,60.928571
2,2,671,2018,3,48.907143
3,3,671,2018,4,52.942857
4,4,671,2018,5,64.071429


In [15]:
madbt.groupby(['year'])["ID"].nunique()

year
2018    175
2019    165
2020    142
2021    156
Name: ID, dtype: int64

In [16]:
madbt["ID"].nunique()

250

In [17]:
# Lets add the variable
madbt1=pd.merge(madbt, bdata1, on=['ID'], how='inner')
madbt1.head()

,Unnamed: 0,ID,year,month,AASHTO_MADT,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,...,Median Age,employment_density,Total Commuter,Bike Commuter,Percentage of Bike Commuter,bike commuter density,Name,name,highway,Bicycle_Fa
0,0,671,2018,1,65.478571,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,38.966406,6219.810255,3127.241241,159.824309,5.110713,203.495292,1,Bay Area Ridge Trail,Path,Path
1,1,671,2018,2,60.928571,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,38.966406,6219.810255,3127.241241,159.824309,5.110713,203.495292,1,Bay Area Ridge Trail,Path,Path
2,2,671,2018,3,48.907143,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,38.966406,6219.810255,3127.241241,159.824309,5.110713,203.495292,1,Bay Area Ridge Trail,Path,Path
3,3,671,2018,4,52.942857,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,38.966406,6219.810255,3127.241241,159.824309,5.110713,203.495292,1,Bay Area Ridge Trail,Path,Path
4,4,671,2018,5,64.071429,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,38.966406,6219.810255,3127.241241,159.824309,5.110713,203.495292,1,Bay Area Ridge Trail,Path,Path


In [18]:
madbt1.shape

(5327, 54)

In [19]:
madbt1["ID"].nunique()

250

In [20]:
# Now add Strava MADBT data
stv=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/Eco_Strava_bike_data_18_21_May_8.csv')
cols=["year","month"]
stv['date'] = stv[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
stv['date'] = pd.to_datetime(stv['date'], errors='coerce')
stv['days_in_month'] = stv['date'].dt.days_in_month
stv['Strava_MADT']=stv['stv_total']/stv['days_in_month']
stv.head()

,Unnamed: 0,ID,latitude,longitude,edgeUID,osmId,edge_uid,osmId.1,month,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,year,date,days_in_month,Strava_MADT
0,0,671,37.79116,-122.45944,308723101.0,215845474.0,308723101,215845474.0,8,420,20,400,0.047619,0.952381,2018,2018-08-01,31,13.548387
1,1,671,37.79116,-122.45944,308723101.0,215845474.0,308723101,215845474.0,12,385,10,375,0.025974,0.974026,2018,2018-12-01,31,12.419355
2,2,671,37.79116,-122.45944,308723101.0,215845474.0,308723101,215845474.0,1,395,25,370,0.063291,0.936709,2018,2018-01-01,31,12.741935
3,3,671,37.79116,-122.45944,308723101.0,215845474.0,308723101,215845474.0,11,270,15,255,0.055556,0.944444,2018,2018-11-01,30,9.000000
4,4,671,37.79116,-122.45944,308723101.0,215845474.0,308723101,215845474.0,4,430,30,400,0.069767,0.930233,2018,2018-04-01,30,14.333333


In [21]:
madbt2=pd.merge(madbt1, stv, on=["ID",'year','month','latitude','longitude'], how='inner')
madbt2.shape


(4540, 67)

In [22]:
madbt2['ID'].nunique()

181

In [23]:
# rest of ID have zero strava Activities
common = madbt1.merge(stv,on=["ID",'year','month','latitude','longitude'])
zero_stv=madbt1[(~madbt1.ID.isin(common.ID))&(~madbt1.ID.isin(common.ID))]
zero_stv.shape

(789, 54)

In [24]:
# Now concate the zero strava sites
madbt3=pd.concat([madbt2, zero_stv],axis=0)
madbt3=madbt3.fillna(0)
madbt3.head()

,Unnamed: 0_x,ID,year,month,AASHTO_MADT,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,...,osmId.1,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,date,days_in_month,Strava_MADT,Unnamed: 0
0,0.0,671,2018,1,65.478571,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,215845474.0,395.0,25.0,370.0,0.063291,0.936709,2018-01-01 00:00:00,31.0,12.741935,0.0
1,1.0,671,2018,2,60.928571,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,215845474.0,465.0,25.0,440.0,0.053763,0.946237,2018-02-01 00:00:00,28.0,16.607143,0.0
2,2.0,671,2018,3,48.907143,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,215845474.0,385.0,10.0,375.0,0.025974,0.974026,2018-03-01 00:00:00,31.0,12.419355,0.0
3,3.0,671,2018,4,52.942857,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,215845474.0,430.0,30.0,400.0,0.069767,0.930233,2018-04-01 00:00:00,30.0,14.333333,0.0
4,4.0,671,2018,5,64.071429,37.79116,-122.45944,POLYGON ((-122.45030116607526 37.7911177158698...,5.15385,0.0,...,215845474.0,475.0,25.0,450.0,0.052632,0.947368,2018-05-01 00:00:00,31.0,15.322581,0.0


In [25]:
madbt3['highway'].unique()

array(['Path', 'cycleway', 'secondary', 'tertiary', 'primary',
       'residential', 'service', 'pedestrian', 'trunk'], dtype=object)

In [26]:
madbt3['Path'] = np.where(madbt3['Bicycle_Fa']=='Path', 1, 0)
madbt3['Bike Route'] = np.where(madbt3['Bicycle_Fa']=='Bike Route', 1, 0)
madbt3['Bike Lane'] = np.where(madbt3['Bicycle_Fa']=='Bike Lane', 1, 0)
madbt3['Cycle Track'] = np.where(madbt3['Bicycle_Fa']=='Cycle Track', 1, 0)
madbt3['Trail'] = np.where(madbt3['Bicycle_Fa']=='Trail', 1, 0)

In [27]:
madbt3['Path'] = np.where(madbt3['highway']=='Path', 1, 0)
madbt3['cycleway'] = np.where(madbt3['highway']=='cycleway', 1, 0)
madbt3['secondary'] = np.where(madbt3['highway']=='secondary', 1, 0)
madbt3['tertiary'] = np.where(madbt3['highway']=='tertiary', 1, 0)
madbt3['primary'] = np.where(madbt3['highway']=='primary', 1, 0)
madbt3['residential'] = np.where(madbt3['highway']=='residential', 1, 0)
madbt3['service'] = np.where(madbt3['highway']=='service', 1, 0)
madbt3['trunk'] = np.where(madbt3['highway']=='trunk', 1, 0)

In [28]:
madbt3.shape, madbt3['ID'].nunique()

((5329, 80), 250)

In [29]:
madbt3=madbt3.drop_duplicates(subset=["ID",'month','year','latitude','longitude'], keep='first')
madbt3.shape

(4291, 80)

In [30]:
madbt3['ID'].nunique()

250

In [31]:
# import Imputation model data
madbt3.to_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/RF_Imputation_model_data/bike_imputation_model_data.csv')

# prepared bike model application data

In [32]:
all_data=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/all_valid_non_valid_bike_counters_data.csv')
non_valid_data=all_data.loc[all_data['days_in_month']>all_data['no_of_days_data_collected']]
non_valid_data=non_valid_data.drop_duplicates(subset=['ID','year','month'], keep='first')
non_valid_data=non_valid_data[['ID','year','month']]
non_valid_data.head(4)

,ID,year,month
151,671,2018,6
729,671,2020,1
972,671,2020,9
973,671,2020,11


In [33]:
non_valid_data['ID'].nunique()

390

In [34]:
# get the sites that was not qualified in QC
common = bdata.merge(madbt,on=['ID'])
nqcb=bdata[(~bdata.ID.isin(common.ID))&(~bdata.ID.isin(common.ID))]
nqcb1=pd.merge(nqcb, bike_fac, on=['ID'], how='inner')
nqcb1.shape

(164, 50)

In [35]:
nqcb2=pd.merge(nqcb1,non_valid_data,on=["ID"], how='inner')
nqcb2.head()

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,Total Commuter,Bike Commuter,Percentage of Bike Commuter,bike commuter density,Name,name,highway,Bicycle_Fa,year,month
0,1127,37.796165,-122.419490,POLYGON ((-122.4103506127677 37.79611961207228...,3.019345,0.002189,0.249510,0.165813,0.131352,0.0,...,11234.019440,197.352495,1.756740,251.277818,2,Broadway,primary,Bike Route,2018,1
1,2415,37.774200,-122.436295,POLYGON ((-122.42715829014239 37.7741559374201...,4.642047,0.000000,0.687274,0.087677,0.155897,0.0,...,16134.096210,1351.356394,8.375780,1720.605996,3,Fell Street,primary,Bike Lane,2018,1
2,2691,35.287736,-120.657826,POLYGON ((-120.64898426571617 35.2875646426652...,76.562372,0.008590,0.397234,0.009334,0.254896,0.0,...,3148.238608,400.995099,12.737125,510.564478,4,Halsey Circle,residential,Path,2018,1
3,2692,35.287736,-120.657826,POLYGON ((-120.64898426571617 35.2875646426652...,76.562372,0.008590,0.397234,0.009334,0.254896,0.0,...,3148.238608,400.995099,12.737125,510.564478,5,Halsey Circle,residential,Path,2018,1
4,2696,33.636400,-117.573600,POLYGON ((-117.5649623942388 33.63601951720258...,30.272075,0.000000,4.698224,0.000000,5.178166,0.0,...,418.449730,0.000000,0.000000,0.000000,8,,cycleway,Trail,2018,1


In [36]:
nqcb2['ID'].nunique()

162

In [37]:
stv.columns

Index(['Unnamed: 0', 'ID', 'latitude', 'longitude', 'edgeUID', 'osmId',
       'edge_uid', 'osmId.1', 'month', 'stv_total', 'stv_com_total',
       'stv_rec_total', 'stv_com_ratio', 'stv_rec_ratio', 'year', 'date',
       'days_in_month', 'Strava_MADT'],
      dtype='object')

In [38]:
# Lets add Strava 
nqcb3=pd.merge(nqcb2,stv, on=['ID','latitude','longitude','year','month'], how='inner')
nqcb3.head()

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,edge_uid,osmId.1,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,date,days_in_month,Strava_MADT
0,1127,37.796165,-122.419490,POLYGON ((-122.4103506127677 37.79611961207228...,3.019345,0.002189,0.249510,0.165813,0.131352,0.000000,...,308779961,87297166.0,25,10,15,0.400000,0.600000,2018-01-01,31,0.806452
1,2415,37.774200,-122.436295,POLYGON ((-122.42715829014239 37.7741559374201...,4.642047,0.000000,0.687274,0.087677,0.155897,0.000000,...,308725144,907910631.0,2200,745,1455,0.338636,0.661364,2018-01-01,31,70.967742
2,2696,33.636400,-117.573600,POLYGON ((-117.5649623942388 33.63601951720258...,30.272075,0.000000,4.698224,0.000000,5.178166,0.000000,...,308341527,252712475.0,20,0,20,0.000000,1.000000,2018-01-01,31,0.645161
3,2735,37.851585,-122.260410,POLYGON ((-122.25126407202679 37.8515272002856...,3.516737,0.008472,0.261220,0.035638,0.030689,0.000676,...,309663180,416105082.0,220,85,135,0.386364,0.613636,2018-01-01,31,7.096774
4,3563,32.991085,-117.271470,POLYGON ((-117.26290023488116 32.9906902025036...,77.836301,0.004015,0.124383,0.014682,0.099479,0.000000,...,305965867,345780741.0,3715,215,3500,0.057873,0.942127,2018-01-01,31,119.838710


In [39]:
nqcb3['ID'].nunique()

85

In [40]:
# add the sites that have strava zero
# rest of ID have zero strava Activities
common = nqcb2.merge(nqcb3,on=['ID','latitude','longitude','year','month'])
zero_ap=nqcb2[(~nqcb2.ID.isin(common.ID))&(~nqcb2.ID.isin(common.ID))]
zero_ap.shape

(134, 52)

In [41]:
# add both
# Now concate the zero strava sites
nqcb4=pd.concat([nqcb3, zero_ap],axis=0)
nqcb4=nqcb4.fillna(0)
nqcb4.head()

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,edge_uid,osmId.1,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,date,days_in_month,Strava_MADT
0,1127,37.796165,-122.419490,POLYGON ((-122.4103506127677 37.79611961207228...,3.019345,0.002189,0.249510,0.165813,0.131352,0.000000,...,308779961.0,87297166.0,25.0,10.0,15.0,0.400000,0.600000,2018-01-01 00:00:00,31.0,0.806452
1,2415,37.774200,-122.436295,POLYGON ((-122.42715829014239 37.7741559374201...,4.642047,0.000000,0.687274,0.087677,0.155897,0.000000,...,308725144.0,907910631.0,2200.0,745.0,1455.0,0.338636,0.661364,2018-01-01 00:00:00,31.0,70.967742
2,2696,33.636400,-117.573600,POLYGON ((-117.5649623942388 33.63601951720258...,30.272075,0.000000,4.698224,0.000000,5.178166,0.000000,...,308341527.0,252712475.0,20.0,0.0,20.0,0.000000,1.000000,2018-01-01 00:00:00,31.0,0.645161
3,2735,37.851585,-122.260410,POLYGON ((-122.25126407202679 37.8515272002856...,3.516737,0.008472,0.261220,0.035638,0.030689,0.000676,...,309663180.0,416105082.0,220.0,85.0,135.0,0.386364,0.613636,2018-01-01 00:00:00,31.0,7.096774
4,3563,32.991085,-117.271470,POLYGON ((-117.26290023488116 32.9906902025036...,77.836301,0.004015,0.124383,0.014682,0.099479,0.000000,...,305965867.0,345780741.0,3715.0,215.0,3500.0,0.057873,0.942127,2018-01-01 00:00:00,31.0,119.838710


In [42]:
check=nqcb4.loc[nqcb4['ID']==58246]
check

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,edge_uid,osmId.1,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,date,days_in_month,Strava_MADT
122,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,5.0,0.0,5.0,0.000000,1.000000,2018-01-01 00:00:00,31.0,0.161290
123,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,20.0,5.0,15.0,0.250000,0.750000,2019-03-01 00:00:00,31.0,0.645161
124,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,510.0,25.0,485.0,0.049020,0.950980,2019-04-01 00:00:00,30.0,17.000000
125,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,5.0,0.0,5.0,0.000000,1.000000,2019-05-01 00:00:00,31.0,0.161290
126,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,215.0,40.0,175.0,0.186047,0.813953,2019-06-01 00:00:00,30.0,7.166667
127,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,385.0,100.0,285.0,0.259740,0.740260,2019-10-01 00:00:00,31.0,12.419355
128,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,20.0,5.0,15.0,0.250000,0.750000,2019-11-01 00:00:00,30.0,0.666667
129,58246,38.94835,-119.95825,POLYGON ((-119.94897720869996 38.9481077526945...,86.767507,0.001602,0.216006,0.000107,0.497021,0.00021,...,311398264.0,500227285.0,315.0,65.0,250.0,0.206349,0.793651,2019-12-01 00:00:00,31.0,10.161290


In [43]:
nqcb4['ID'].nunique()

162

In [44]:
nqcb4.columns

Index(['ID', 'latitude', 'longitude', 'geometry', 'min_dist_to_CBD',
       'Commercial Area ', 'Distance to commercial area', 'retail Area ',
       'Distance to retail area', 'industrial Area ',
       'Distance to industrial area', 'residential Area ',
       'Distance to residential area', 'grass Area ', 'Distance to grass area',
       'forest Area ', 'Distance to forest area', 'education Area ',
       'Distance to education area', 'institutional Area ',
       'Distance to institutional area', 'water Area',
       'Distance to water area', 'park Area ', 'Distance to Park Center',
       'School', 'min_dist_to_school', 'college', 'min_dist_to_college',
       'University', 'min_dist_to_university', 'pct_white',
       'pct_African_American', 'pct_male', 'pct_female', 'Student Access',
       'pct_at_least_college_education', 'HH_density', 'Median_HH_income',
       'population_density', 'Median Age', 'employment_density',
       'Total Commuter', 'Bike Commuter', 'Percentage of B

In [45]:
nqcb4=nqcb4.drop_duplicates(subset=["ID",'month','year','latitude','longitude'], keep='first')
nqcb4.shape

(266, 65)

In [46]:
nqcb4['ID'].nunique()

162

In [47]:
# import Imputation model data
nqcb3.to_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/RF_Imputation_model_data/bike_imputation_model_application_data.csv')

# Process Pedestrain data

In [48]:
# Read ped counter variables data
pdata=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/Eco_CA_Ped_data_circular_0.25_mile_buffer.csv')
pdata.head()

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,pct_at_least_college_education,HH_density,Median_HH_income,population_density,Median Age,employment_density,Total Commuter,Walk Commuter,Percentage of Walk Commuter,Walk commuter density
0,572,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,1.001268,0.0,0.586396,0.0,...,91.947499,559.508075,208000.000000,1152.823284,30.200000,931.809142,138.319911,8.546461,6.178764,43.526911
1,588,37.793898,-122.455561,POLYGON ((-122.45099097999639 37.7938764145384...,4.889865,0.0,0.515058,0.0,0.655687,0.0,...,92.279192,592.390519,210739.958418,1297.035477,32.438325,1026.436451,149.147696,8.721225,5.847375,44.416978
2,589,37.793898,-122.455561,POLYGON ((-122.45099097999639 37.7938764145384...,4.889865,0.0,0.515058,0.0,0.655687,0.0,...,92.279192,592.390519,210739.958418,1297.035477,32.438325,1026.436451,149.147696,8.721225,5.847375,44.416978
3,595,37.792583,-122.458150,POLYGON ((-122.45358049399331 37.7925618958346...,5.053731,0.0,0.678132,0.0,0.721444,0.0,...,93.688703,846.104922,219501.087641,2392.231666,40.054783,1769.674912,235.847198,10.058299,4.264752,51.226661
4,671,37.791160,-122.459440,POLYGON ((-122.45487058064846 37.7911389465364...,5.153850,0.0,0.797617,0.0,0.818882,0.0,...,92.301012,1817.953749,182448.977408,5413.545187,40.152337,4314.354911,549.884362,19.808159,3.602241,100.882451


In [49]:
ped_fac=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/Facilities_data/CA_Ped_Facilities- Ped_manual_facilities.csv')
ped_fac=ped_fac.drop(['latitude','longitude'], axis=1)
ped_fac.head()

,Name,ID,Pedestrain Facilities,highway
0,1,572,Trail,Path
1,2,588,Path,residential
2,4,595,Trail,Path
3,5,671,Path,Path
4,6,780,Trail,Path


In [50]:
ped_fac['Pedestrain Facilities'].unique()

array(['Trail', 'Path', 'Bike Lane', 'Cycle Track', 'cycle track',
       'Bike Route', 'trail'], dtype=object)

In [51]:
pdata1=pd.merge(pdata, ped_fac, on=['ID'], how='inner')
pdata1.shape

(282, 49)

In [52]:
# add our MADPT after QC
madpt=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/our_ped_AASHTO_MADT for CA.csv')
madpt.head()

,Unnamed: 0,ID,year,month,AASHTO_MADT
0,0,572,2018,1,188.971429
1,1,572,2018,2,213.142857
2,2,572,2018,3,235.550000
3,3,572,2018,4,221.250000
4,4,572,2018,5,220.435714


In [53]:
madpt.shape

(3922, 5)

In [54]:
madpt.groupby(['year'])["ID"].nunique()

year
2018     90
2019     88
2020     92
2021    123
2022    145
Name: ID, dtype: int64

In [55]:
madpt["ID"].nunique()

208

In [56]:
# Lets add the variable
madpt1=pd.merge(madpt, pdata1, on=['ID'], how='inner')
madpt1.head()

,Unnamed: 0,ID,year,month,AASHTO_MADT,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,...,population_density,Median Age,employment_density,Total Commuter,Walk Commuter,Percentage of Walk Commuter,Walk commuter density,Name,Pedestrain Facilities,highway
0,0,572,2018,1,188.971429,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,1152.823284,30.2,931.809142,138.319911,8.546461,6.178764,43.526911,1,Trail,Path
1,1,572,2018,2,213.142857,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,1152.823284,30.2,931.809142,138.319911,8.546461,6.178764,43.526911,1,Trail,Path
2,2,572,2018,3,235.550000,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,1152.823284,30.2,931.809142,138.319911,8.546461,6.178764,43.526911,1,Trail,Path
3,3,572,2018,4,221.250000,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,1152.823284,30.2,931.809142,138.319911,8.546461,6.178764,43.526911,1,Trail,Path
4,4,572,2018,5,220.435714,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,1152.823284,30.2,931.809142,138.319911,8.546461,6.178764,43.526911,1,Trail,Path


In [57]:
# Now add Strava MADBT data
ped_stv=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/ped_eco_strava_monthly_matching2018_2022.csv')
cols=["year","month"]
ped_stv['date'] = ped_stv[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
ped_stv['date'] = pd.to_datetime(ped_stv['month'], errors='coerce')
ped_stv['month'] = ped_stv['date'].dt.month
ped_stv['year'] = ped_stv['date'].dt.year
ped_stv['days_in_month'] = ped_stv['date'].dt.days_in_month
ped_stv['Strava_MADT']=ped_stv['stv_total']/stv['days_in_month']
ped_stv.head()

,Unnamed: 0,ID,latitude,longitude,Pedestrain Facilities,highway,edgeUID,osmId,dist,edge_uid,...,osm_reference_id,year,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,date,days_in_month,Strava_MADT
0,0,572,37.798027,-122.467575,Trail,Path,308772257.0,133469908.0,d4,308772257,...,133469908,2018,635,55,580,0.086614,0.913386,2018-08-01,31,20.483871
1,1,572,37.798027,-122.467575,Trail,Path,308772257.0,133469908.0,d4,308772257,...,133469908,2018,455,25,430,0.054945,0.945055,2018-12-01,31,14.677419
2,2,572,37.798027,-122.467575,Trail,Path,308772257.0,133469908.0,d4,308772257,...,133469908,2018,535,35,500,0.065421,0.934579,2018-01-01,31,17.258065
3,3,572,37.798027,-122.467575,Trail,Path,308772257.0,133469908.0,d4,308772257,...,133469908,2018,290,10,280,0.034483,0.965517,2018-11-01,30,9.666667
4,4,572,37.798027,-122.467575,Trail,Path,308772257.0,133469908.0,d4,308772257,...,133469908,2018,545,50,495,0.091743,0.908257,2018-04-01,30,18.166667


In [58]:
madpt1['month'].dtypes,ped_stv['month'].dtypes

(dtype('int64'), dtype('int64'))

In [59]:
ped_data2=pd.merge(madpt1, ped_stv, on=["ID",'year','month','latitude','longitude'], how='inner')
ped_data2.head()

,Unnamed: 0_x,ID,year,month,AASHTO_MADT,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,...,reverse_average_speed,osm_reference_id,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,date,days_in_month,Strava_MADT
0,0,572,2018,1,188.971429,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,2.79,133469908,535,35,500,0.065421,0.934579,2018-01-01,31,17.258065
1,1,572,2018,2,213.142857,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,2.48,133469908,520,50,470,0.096154,0.903846,2018-02-01,28,18.571429
2,2,572,2018,3,235.550000,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,3.02,133469908,615,40,575,0.065041,0.934959,2018-03-01,31,19.838710
3,3,572,2018,4,221.250000,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,3.01,133469908,545,50,495,0.091743,0.908257,2018-04-01,30,18.166667
4,4,572,2018,5,220.435714,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,2.89,133469908,600,45,555,0.075000,0.925000,2018-05-01,31,19.354839


In [60]:
ped_data2['ID'].nunique()

78

In [61]:
# rest of ID have zero strava Activities
common = madpt1.merge(ped_stv,on=["ID",'year','month','latitude','longitude'])
zero_stvp=madpt1[(~madpt1.ID.isin(common.ID))&(~madpt1.ID.isin(common.ID))]
zero_stvp.shape

(1881, 53)

In [62]:
# Now concate the zero strava sites
madpt3=pd.concat([ped_data2, zero_stvp],axis=0)
madpt3=madpt3.fillna(0)
madpt3.head()

,Unnamed: 0_x,ID,year,month,AASHTO_MADT,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,...,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,date,days_in_month,Strava_MADT,Unnamed: 0,Pedestrain Facilities,highway
0,0.0,572,2018,1,188.971429,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,35.0,500.0,0.065421,0.934579,2018-01-01 00:00:00,31.0,17.258065,0.0,0,0
1,1.0,572,2018,2,213.142857,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,50.0,470.0,0.096154,0.903846,2018-02-01 00:00:00,28.0,18.571429,0.0,0,0
2,2.0,572,2018,3,235.550000,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,40.0,575.0,0.065041,0.934959,2018-03-01 00:00:00,31.0,19.838710,0.0,0,0
3,3.0,572,2018,4,221.250000,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,50.0,495.0,0.091743,0.908257,2018-04-01 00:00:00,30.0,18.166667,0.0,0,0
4,4.0,572,2018,5,220.435714,37.798027,-122.467575,POLYGON ((-122.46300515156133 37.7980062588424...,5.437166,0.0,...,45.0,555.0,0.075000,0.925000,2018-05-01 00:00:00,31.0,19.354839,0.0,0,0


In [63]:
madpt3['ID'].nunique()

208

In [64]:
madpt3['Path'] = np.where(madpt3['Pedestrain Facilities']=='Path', 1, 0)
madpt3['Bike Route'] = np.where(madpt3['Pedestrain Facilities']=='Bike Route', 1, 0)
madpt3['Bike Lane'] = np.where(madpt3['Pedestrain Facilities']=='Bike Lane', 1, 0)
madpt3['Cycle Track'] = np.where(madpt3['Pedestrain Facilities']=='Cycle Track', 1, 0)
madpt3['Trail'] = np.where(madpt3['Pedestrain Facilities']=='Trail', 1, 0)
madpt3['Path'] = np.where(madpt3['highway']=='Path', 1, 0)
madpt3['cycleway'] = np.where(madpt3['highway']=='cycleway', 1, 0)
madpt3['secondary'] = np.where(madpt3['highway']=='secondary', 1, 0)
madpt3['tertiary'] = np.where(madpt3['highway']=='tertiary', 1, 0)
madpt3['primary'] = np.where(madpt3['highway']=='primary', 1, 0)
madpt3['residential'] = np.where(madpt3['highway']=='residential', 1, 0)
madpt3['service'] = np.where(madpt3['highway']=='service', 1, 0)
madpt3['trunk'] = np.where(madpt3['highway']=='trunk', 1, 0)

In [65]:
madpt3=madpt3.drop_duplicates(subset=["ID",'month','year','latitude','longitude'], keep='first')
madpt3.shape

(3788, 118)

In [66]:
madpt3['ID'].nunique()

208

In [67]:
# import Imputation model data
madpt3.to_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/RF_Imputation_model_data/ped_imputation_model_data.csv')

# prepared Ped model application data

In [68]:
all_pdata=pd.read_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/all_valid_non_valid_ped_counters_data.csv')
non_valid_pdata=all_pdata.loc[all_pdata['days_in_month']>all_pdata['no_of_days_data_collected']]
non_valid_pdata=non_valid_pdata.drop_duplicates(subset=['ID','year','month'], keep='first')
non_valid_pdata=non_valid_pdata[['ID','year','month']]
non_valid_pdata.head(4)

,ID,year,month
1430,572,2021,12
1460,572,2022,1
1461,572,2022,2
1779,588,2018,1


In [69]:
non_valid_pdata['ID'].nunique()

282

In [70]:
# get the sites that was not qualified in QC
common = pdata.merge(madpt,on=['ID'])
nqcp=pdata[(~pdata.ID.isin(common.ID))&(~pdata.ID.isin(common.ID))]
nqcp1=pd.merge(nqcp, ped_fac, on=['ID'], how='inner')
nqcp1.shape

(74, 49)

In [71]:
nqcp2=pd.merge(nqcp1,non_valid_pdata,on=["ID"], how='inner')
nqcp2.head()

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,employment_density,Total Commuter,Walk Commuter,Percentage of Walk Commuter,Walk commuter density,Name,Pedestrain Facilities,highway,year,month
0,588,37.793898,-122.455561,POLYGON ((-122.45099097999639 37.7938764145384...,4.889865,0.0,0.515058,0.0,0.655687,0.0,...,1026.436451,149.147696,8.721225,5.847375,44.416978,2,Path,residential,2018,1
1,589,37.793898,-122.455561,POLYGON ((-122.45099097999639 37.7938764145384...,4.889865,0.0,0.515058,0.0,0.655687,0.0,...,1026.436451,149.147696,8.721225,5.847375,44.416978,3,Path,residential,2018,1
2,780,37.798060,-122.467319,POLYGON ((-122.46274951901897 37.7980395200501...,5.423048,0.0,0.987122,0.0,0.573773,0.0,...,931.809142,138.319911,8.546461,6.178764,43.526911,6,Trail,Path,2018,1
3,781,37.798060,-122.467319,POLYGON ((-122.46274951901897 37.7980395200501...,5.423048,0.0,0.987122,0.0,0.573773,0.0,...,931.809142,138.319911,8.546461,6.178764,43.526911,7,Trail,Path,2018,1
4,2696,33.636400,-117.573600,POLYGON ((-117.56928117684645 33.6362098335502...,30.272075,0.0,4.980704,0.0,5.164795,0.0,...,644.715602,74.938867,0.000005,0.000007,0.000026,12,Trail,tertiary,2018,1


In [72]:
nqcp2['ID'].nunique()

74

In [73]:
# Lets add Strava 
nqcp3=pd.merge(nqcp2,ped_stv, on=['ID','latitude','longitude','year','month'], how='inner')
nqcp3.head()

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,reverse_average_speed,osm_reference_id,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,date,days_in_month,Strava_MADT
0,2696,33.63640,-117.57360,POLYGON ((-117.56928117684645 33.6362098335502...,30.272075,0.0,4.980704,0.0,5.164795,0.0,...,2.15,13280951,45,0,45,0.0,1.0,2018-01-01,31,1.451613
1,2698,33.80296,-117.76664,POLYGON ((-117.76231146249543 33.8027757810293...,19.164520,0.0,5.609606,0.0,5.510134,0.0,...,3.80,787298838,10,0,10,0.0,1.0,2018-01-01,31,0.322581
2,2698,33.80296,-117.76664,POLYGON ((-117.76231146249543 33.8027757810293...,19.164520,0.0,5.609606,0.0,5.510134,0.0,...,2.93,787298838,105,0,105,0.0,1.0,2018-02-01,28,3.750000
3,2698,33.80296,-117.76664,POLYGON ((-117.76231146249543 33.8027757810293...,19.164520,0.0,5.609606,0.0,5.510134,0.0,...,3.02,787298838,55,0,55,0.0,1.0,2018-03-01,31,1.774194
4,2698,33.80296,-117.76664,POLYGON ((-117.76231146249543 33.8027757810293...,19.164520,0.0,5.609606,0.0,5.510134,0.0,...,2.54,787298838,45,0,45,0.0,1.0,2018-04-01,30,1.500000


In [74]:
# add the sites that have strava zero
# rest of ID have zero strava Activities
common = nqcp2.merge(nqcp3,on=['ID','latitude','longitude','year','month'])
zero_app=nqcp2[(~nqcp2.ID.isin(common.ID))&(~nqcp2.ID.isin(common.ID))]
zero_app.shape

(102, 51)

In [75]:
# add both
# Now concate the zero strava sites
nqcp4=pd.concat([nqcp3, zero_app],axis=0)
nqcp4=nqcp4.fillna(0)
nqcp4.head()

,ID,latitude,longitude,geometry,min_dist_to_CBD,Commercial Area,Distance to commercial area,retail Area,Distance to retail area,industrial Area,...,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,date,days_in_month,Strava_MADT,Pedestrain Facilities,highway
0,2696,33.63640,-117.57360,POLYGON ((-117.56928117684645 33.6362098335502...,30.272075,0.0,4.980704,0.0,5.164795,0.0,...,45.0,0.0,45.0,0.0,1.0,2018-01-01 00:00:00,31.0,1.451613,0,0
1,2698,33.80296,-117.76664,POLYGON ((-117.76231146249543 33.8027757810293...,19.164520,0.0,5.609606,0.0,5.510134,0.0,...,10.0,0.0,10.0,0.0,1.0,2018-01-01 00:00:00,31.0,0.322581,0,0
2,2698,33.80296,-117.76664,POLYGON ((-117.76231146249543 33.8027757810293...,19.164520,0.0,5.609606,0.0,5.510134,0.0,...,105.0,0.0,105.0,0.0,1.0,2018-02-01 00:00:00,28.0,3.750000,0,0
3,2698,33.80296,-117.76664,POLYGON ((-117.76231146249543 33.8027757810293...,19.164520,0.0,5.609606,0.0,5.510134,0.0,...,55.0,0.0,55.0,0.0,1.0,2018-03-01 00:00:00,31.0,1.774194,0,0
4,2698,33.80296,-117.76664,POLYGON ((-117.76231146249543 33.8027757810293...,19.164520,0.0,5.609606,0.0,5.510134,0.0,...,45.0,0.0,45.0,0.0,1.0,2018-04-01 00:00:00,30.0,1.500000,0,0


In [76]:
nqcp4['Path'] = np.where(nqcp4['Pedestrain Facilities']=='Path', 1, 0)
nqcp4['Bike Route'] = np.where(nqcp4['Pedestrain Facilities']=='Bike Route', 1, 0)
nqcp4['Bike Lane'] = np.where(nqcp4['Pedestrain Facilities']=='Bike Lane', 1, 0)
nqcp4['Cycle Track'] = np.where(nqcp4['Pedestrain Facilities']=='Cycle Track', 1, 0)
nqcp4['Trail'] = np.where(nqcp4['Pedestrain Facilities']=='Trail', 1, 0)
nqcp4['Path'] = np.where(nqcp4['highway']=='Path', 1, 0)
nqcp4['cycleway'] = np.where(nqcp4['highway']=='cycleway', 1, 0)
nqcp4['secondary'] = np.where(nqcp4['highway']=='secondary', 1, 0)
nqcp4['tertiary'] = np.where(nqcp4['highway']=='tertiary', 1, 0)
nqcp4['primary'] = np.where(nqcp4['highway']=='primary', 1, 0)
nqcp4['residential'] = np.where(nqcp4['highway']=='residential', 1, 0)
nqcp4['service'] = np.where(nqcp4['highway']=='service', 1, 0)
nqcp4['trunk'] = np.where(nqcp4['highway']=='trunk', 1, 0)

In [77]:
nqcp4=nqcp4.drop_duplicates(subset=["ID",'month','year','latitude','longitude'], keep='first')
nqcp4.shape

(198, 115)

In [78]:
# import Imputation model data
nqcp4.to_csv('C:/UC Berkeley Project Backup/AT Census/Lindsey _QA/QC Article Analysis/Variables/RF_Imputation_model_data/ped_imputation_model_application_data.csv')

# end of the code